# Ready, Steady, Go AI (*Advanced Exercises*)

This notebook is a supplement to the paper, **Ready, Steady, Go AI: A Practical Tutorial on Explainable Artificial Intelligence and Its Applications in Phenomics Image Analysis** (submitted to *Patterns, 2021*) by Farid Nakhle and Antoine Harfouche

Read the accompanying paper [here](https://doi.org).

If you have any questions or feedback regarding this tutorial, please contact Farid Nakhle <<farid.nakhle@gmail.com>> or Antoine Harfouche <<aharfouche@unitus.it>>.

# Table of Contents


* **1. Introduction**
* **2. Exercise I: Splitting Data**
* **3. Exercise II: Cropping Apple Leaf Images Using a YOLO Model Pretrained on Tomato Leaves**
* **4. Exercise III: Segmenting Apple Leaf Images Using a SegNet Model Pretrained on Tomato Leaves**
* **5. Exercise IV: Descriptive Data Analysis**
* **6. Exercise V: Balancing the Dataset**
* **7. Exercise VI: Training and Classifying Using the DenseNet-161 Pretrained DCNN algorithm**
* **8. Exercise VII: Generating Confusion Matrix**
* **9. Exercise VIII: Generating Explanations With LIME (Quickshift and Compact-Watershed)**

# 1. Introduction


Before attempting to resolve the exercises found in this notebook, visit our Github repository and try to open and run all the notebooks provided by the tutorial. 

Here, the solution for each exercise can be found in a hidden code cell at its end.

Users should try to solve the exercises with the help of the notebooks provided by the tutorial before looking at the solution.

As a reminder, we are working with the PlantVillage dataset, originally obtained from [here](http://dx.doi.org/10.17632/tywbtsjrjv.1).
For the following exercises, we will be working with a subset of PlantVillage containing the apple classes only. We have made the subset available [here](http://faridnakhle.com/pv/PlantVillage_Apple.zip). 

**It is important to note that Colab deletes all unsaved data once the instance is recycled. Therefore, remember to download your results once you run the code.**

#2. Exercise I: Data Splitting


**A.** Write a code that uses HTTP requests to download the PlantVillage apple leaves dataset available on the link provided in the introduction. The dataset must be saved then extracted to /content/dataset/original/.

**B.** Use pip to install split-folders, then write a commant to split the dataset into training, validation, and testing sets. Use the following split ratio: training: 60%, validation: 20%, testing: 20%. The split dataset must be saved under /content/dataset/split/

In [ ]:
import requests
import os
import zipfile

############################
### Start Your Code Here ###
############################



## Hint 1: ##
## Google Colab allows you to use the symbol "!" to execute commands from the underlying operating system.
## Example: To use pip, you can write !pip install

## Hint 2: ##
## The split-folders parameters and usage instructions can be found here: https://pypi.org/project/split-folders/ ##

# Solution

In [ ]:
!rm -R /content/dataset/original/
!rm -R /content/dataset/split/

import requests
import os
import zipfile

dataset_url = "http://faridnakhle.com/pv/PlantVillage_Apple.zip"
save_data_to = "/content/dataset/original/"
dataset_file_name = "dataset.zip"

if not os.path.exists(save_data_to):
    os.makedirs(save_data_to)

r = requests.get(dataset_url, stream = True, headers={"User-Agent": "Ready, Steady, Go AI"})

print("Downloading dataset...")  

with open(save_data_to + dataset_file_name, "wb") as file: 
    for block in r.iter_content(chunk_size = 1024):
         if block: 
             file.write(block)

## Extract downloaded zip dataset file
print("Dataset downloaded")  
print("Extracting files...")  
with zipfile.ZipFile(save_data_to + dataset_file_name, 'r') as zip_dataset:
    zip_dataset.extractall(save_data_to)

## Delete the zip file as we no longer need it
os.remove(save_data_to + dataset_file_name)
print("All done!")  

## SPLIT 
!pip install split-folders tqdm
!splitfolders --output "/content/dataset/split/" --seed 1337 --ratio .8 .1 .1 -- "/content/dataset/original"

# 3. Exercise II: Cropping Apple Leaf Images Using a YOLO Model Pretrained on Tomato Leaves

Before you start, make sure to run the "Install and import prequisites", "Download pretrained model", and "Define prerequisite functions" code cells.

This exercise uses the YOLO model previously trained on cropping tomato leaf images to crop the apple leaf images.

You are required to write the cropping function and to modify the code for it to save the cropped images under /content/dataset/cropped/

After that, you should be able to run the "Preview a Cropped Image" code cell and see a sample image from the results.

In [ ]:
#@title Install and import prerequisites
!git clone https://github.com/ultralytics/yolov3
%cd yolov3
%pip install -qr requirements.txt 
import torch
from IPython.display import Image


In [ ]:
#@title Download pretrained model

model_URL = "http://faridnakhle.com/pv/models/YOLOv3.zip"
save_data_to = "/content/models/"
model_file_name = "yolo.zip"

if not os.path.exists(save_data_to):
    os.makedirs(save_data_to)

print("Downloading model...")  

r = requests.get(model_URL, stream = True, headers={"User-Agent": "Ready, Steady, Go AI"})
with open(save_data_to + model_file_name, "wb") as file: 
    for block in r.iter_content(chunk_size = 1024):
         if block: 
             file.write(block)

## Extract downloaded zip dataset file
print("Model downloaded")  
print("Extracting files...")

with zipfile.ZipFile(save_data_to + model_file_name, 'r') as zip_dataset:
    zip_dataset.extractall(save_data_to)
print("All done!")  

In [ ]:
#@title Define prerequisite functions
import os
import cv2
import random
import numpy as np


def plot_grid(img, line_color=(0, 255, 0), thickness=1, type_=cv2.LINE_AA, pxstep=20, pystep=20):
    x = pystep
    y = pxstep

    while x < img.shape[1]:
        cv2.line(img, (x, 0), (x, img.shape[0]), color=line_color, lineType=type_, thickness=thickness)
        x += pystep

    while y < img.shape[0]:
        cv2.line(img, (0, y), (img.shape[1], y), color=line_color, lineType=type_, thickness=thickness)
        y += pxstep
def plot_borders(img, line_color=(0, 255, 0), thickness=1):
    cv2.rectangle(img,(0 ,0),(img.shape[1]-thickness,img.shape[0]-thickness), line_color, thickness)

def myround(x, base=5):
    return base * round(x/base)
def plot_overlay(x, img, color, alpha,
 pxstep=20, pystep=20):
    overlay = img.copy()
    x0, x1, x2, x3 = int(x[0]), int(x[1]), int(x[2]), int(x[3])

    x0 = myround(x0,pystep)
    x1 = myround(x1,pxstep)
    x2 = myround(x2,pystep)
    x3 = myround(x3,pxstep)

    c1, c2 = (x0, x1), (x2, x3)
    cv2.rectangle(overlay, c1, c2, color, -1)
    # apply the overlay
    cv2.addWeighted(overlay, alpha, img, 1 - alpha, 0, img)




!cd /content/yolov3/
import argparse
import time
from pathlib import Path

import cv2
import torch
import torch.backends.cudnn as cudnn
from numpy import random

from models.experimental import attempt_load
from utils.datasets import LoadStreams, LoadImages
from utils.general import check_img_size, non_max_suppression, apply_classifier, scale_coords, xyxy2xywh, \
    strip_optimizer, set_logging, increment_path
from utils.plots import plot_one_box
from utils.torch_utils import select_device, load_classifier, time_synchronized

import glob


In the next code cell, you need to modify the crop_object function so it can crop images based on the detected bounding box by YOLO.

Next, in the crop function, you must modify the code so that the cropped images are saved in /content/dataset/cropped/

In [ ]:
def crop_object(img, coords, img_path):
    # get box coords
    xmin = int(coords[0])
    ymin = int(coords[1])
    xmax = int(coords[2])
    ymax = int(coords[3])
    # WRITE A CODE HERE TO CROP THE IMAGE. 
    # HINT: THE IMAGE IS IN THE "img" VARIABLE
    
    #WRITE A CODE TO SAVE THE IMAGE USING cv2
    # SAVE THE IMAGE TO THE PATH STORED IN "img_path"


"""
YOU NEED TO MODIFY THIS FUNCTION IN ORDER TO SAVE THE CROPPED IMAGE
UNDER /content/dataset/cropped/
"""
def crop(dataset_dir='', model_path='/content/yolov3/runs/train/exp/best.pt'):
    save_txt, imgsz = False, 224
    weights = model_path
    projectP = 'runs/detect'
    projectNameP = 'exp'
    save_img = True
    view_img = True

    save_dir = Path(increment_path(Path(projectP) / projectNameP, False))  # increment run
    (save_dir / 'labels' if save_txt else save_dir).mkdir(parents=True, exist_ok=True)  # make dir

    #loop over train, val, and test set
    trainTestVarDirs = glob.glob(dataset_dir + "*")
    for setDir in trainTestVarDirs:
      splitDir = os.path.basename(setDir)
      setClasses = glob.glob(setDir + "/*")
      for setClass in setClasses:
        # Directories
        classDir = os.path.basename(setClass)
        finalSaveDir = os.path.join(save_dir, splitDir, classDir)
        Path(finalSaveDir).mkdir(parents=True, exist_ok=True)
        source = setClass

        # Initialize
        set_logging()
        device = select_device('0')
        half = device.type != 'cpu'  # half precision only supported on CUDA

        # Load model
        model = attempt_load(weights, map_location=device)  # load FP32 model
        imgsz = check_img_size(imgsz, s=model.stride.max())  # check img_size
        
        #introducing grid size
        gs = model.stride.max()
        #end

        if half:
            model.half()  # to FP16

        # Second-stage classifier
        classify = False
        if classify:
            modelc = load_classifier(name='resnet101', n=2)  # initialize
            modelc.load_state_dict(torch.load('weights/resnet101.pt', map_location=device)['model']).to(device).eval()

        # Set Dataloader
        vid_path, vid_writer = None, None
        
        dataset = LoadImages(source, img_size=imgsz)

        # Get names and colors
        names = model.module.names if hasattr(model, 'module') else model.names
        colors = [[random.randint(0, 255) for _ in range(3)] for _ in names]

        colors = [[217, 175, 78]]

        # Run inference
        t0 = time.time()
        img = torch.zeros((1, 3, imgsz, imgsz), device=device)  # init img
        _ = model(img.half() if half else img) if device.type != 'cpu' else None  # run once
        for path, img, im0s, vid_cap in dataset:
            img = torch.from_numpy(img).to(device)
            img = img.half() if half else img.float()  # uint8 to fp16/32
            img /= 255.0  # 0 - 255 to 0.0 - 1.0
            if img.ndimension() == 3:
                img = img.unsqueeze(0)

            # Inference
            t1 = time_synchronized()
            pred = model(img, augment=True)[0]

            # Apply NMS
            final_pred = non_max_suppression(pred, 0.15, 0.3, classes=0, agnostic=True)
            pred = non_max_suppression(pred, 0.00005, 1, classes=0, agnostic=True)
            t2 = time_synchronized()

            # Apply Classifier
            if classify:
                pred = apply_classifier(pred, modelc, img, im0s)

            # Process detections
            for i, det in enumerate(pred):  # detections per image
                
                p, s, im0 = Path(path), '', im0s.copy()

                imoriginal = im0.copy()
                #plot grid
                numofsquares = int(imgsz/int(gs))
                rowstep = int(im0.shape[0]/numofsquares)
                colstep = int(im0.shape[1]/numofsquares)
                plot_borders(im0, line_color=(0,0,0), thickness=2)
                gridim_solo = im0.copy()
                plot_grid(gridim_solo, pxstep=rowstep, pystep=colstep, line_color=(0,0,0), thickness=2)
                #end plot grid
     
                save_path = str(finalSaveDir + "/" + p.name)
                s += '%gx%g ' % img.shape[2:]  # print string
                gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # normalization gain whwh
                if len(det):
                    # Rescale boxes from img_size to im0 size
                    det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round()

                    # Print results
                    for c in det[:, -1].unique():
                        n = (det[:, -1] == c).sum()  # detections per class
                        s += '%g %ss, ' % (n, names[int(c)])  # add to string
                    
                    # Write results
                    for *xyxy, conf, cls in reversed(det):
                        if save_img or view_img:  # Add bbox to image
                            label = ''#'%s %.2f' % (names[int(cls)], conf)
                            plot_one_box(xyxy, im0, label=label, color=colors[int(cls)], line_thickness=1)
                            
                

                # Print time (inference + NMS)
                print('%sDone. (%.3fs)' % (s, t2 - t1))

                # Save results (image with detections)
                if save_img:
                    cv2.imwrite(save_path + "_original.jpg", imoriginal)
                    cv2.imwrite(save_path, im0)
                    cv2.imwrite(save_path + "_grid.jpg", gridim_solo)
                        


            # SAVE FINAL CROPPED IMAGES
            # Process detections
            for i, det in enumerate(final_pred):  # detections per image
                
                p, s, im0 = Path(path), '', im0s
                im2 = im0.copy() #to use with grid/map
                #background
                numofsquares = int(imgsz/int(gs))
                rowstep = int(im0.shape[0]/numofsquares)
                colstep = int(im0.shape[1]/numofsquares)
                plot_overlay([0,0, im2.shape[1], im2.shape[0]], im2, color=(255, 255, 255), alpha=0.7, pxstep=rowstep, pystep=colstep)
                
                #borders
                plot_borders(im2, line_color=(0,0,0), thickness=2)
                plot_borders(im0, line_color=(0,0,0), thickness=2)

                save_path = str(finalSaveDir + "/" +  p.name)
                s += '%gx%g ' % img.shape[2:]  # print string
                gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # normalization gain whwh
                if len(det):
                    # Rescale boxes from img_size to im0 size
                    det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round()

                    # Print results
                    for c in det[:, -1].unique():
                        n = (det[:, -1] == c).sum()  # detections per class
                        s += '%g %ss, ' % (n, names[int(c)])  # add to string

                    #FUNCTION custom crop
                    CROP = True
                    if CROP:
                        fidx = 0
                        for *xyxy, conf, cls in reversed(det):
                            if save_img or view_img:
                                fidx = fidx + 1
                                crop_object(im0, xyxy, str(finalSaveDir + "/" +  (p.stem + "_cropped_" + str(fidx) + p.suffix)))
                    #END
                    
                    # Write results
                    for *xyxy, conf, cls in reversed(det):
                        if save_img or view_img:  # Add bbox to image
                            label = ''#'%s %.2f' % (names[int(cls)], conf)
                            plot_one_box(xyxy, im0, label=label, color=colors[int(cls)], line_thickness=2)
                            plot_overlay(xyxy, im2, color=colors[int(cls)], alpha=0.7, pxstep=rowstep, pystep=colstep)
                else:
                    cv2.imwrite(save_path + "_not_cropped.jpg", im0)


                gridim = im2.copy()
                plot_grid(gridim, pxstep=rowstep, pystep=colstep, line_color=(0,0,0), thickness=2)
                
                # Print time (inference + NMS)
                print('%sDone. (%.3fs)' % (s, t2 - t1))

                # Save results (image with detections)
                if save_img:
                    cv2.imwrite(save_path + "_map.jpg", gridim)
                    cv2.imwrite(save_path + "_final.jpg", im0)

        if save_txt or save_img:
            s = f"\n{len(list(finalSaveDir.glob('labels/*.txt')))} labels saved to {finalSaveDir + '/' + 'labels'}" if save_txt else ''
            print(f"Results saved to {finalSaveDir}{s}")

        print('Done. (%.3fs)' % (time.time() - t0))

In [ ]:
# REPLACE ? with your answer
crop(dataset_dir=?, model_path='/content/models/weights/RSGAI_YOLOv3.pt')

# Solution

In [ ]:
import os
import cv2
import random
import numpy as np

def crop_object(img, coords, img_path):
    # get box coords
    xmin = int(coords[0])
    ymin = int(coords[1])
    xmax = int(coords[2])
    ymax = int(coords[3])
    # crop detection from image
    cropped_img = img[ymin:ymax, xmin:xmax]
    # save image
    cv2.imwrite(img_path, cropped_img)

def crop(dataset_dir='', model_path='/content/yolov3/runs/train/exp/best.pt'):
    save_txt, imgsz = False, 224
    weights = model_path
    projectP = 'runs/detect'
    projectNameP = 'exp'
    save_img = True
    view_img = True

    save_dir = Path(increment_path(Path(projectP) / projectNameP, False))  # increment run
    (save_dir / 'labels' if save_txt else save_dir).mkdir(parents=True, exist_ok=True)  # make dir
    cropped_save_dir = Path('/content/dataset/cropped')
    cropped_save_dir.mkdir(parents=True, exist_ok=True)  # make dir

    #loop over train, val, and test set
    trainTestVarDirs = glob.glob(dataset_dir + "*")
    for setDir in trainTestVarDirs:
      splitDir = os.path.basename(setDir)
      setClasses = glob.glob(setDir + "/*")
      for setClass in setClasses:
        # Directories
        classDir = os.path.basename(setClass)
        finalSaveDir = os.path.join(save_dir, splitDir, classDir)
        Path(finalSaveDir).mkdir(parents=True, exist_ok=True)
        croppedSaveDir = os.path.join(cropped_save_dir, splitDir, classDir)
        Path(croppedSaveDir).mkdir(parents=True, exist_ok=True)
        source = setClass
        

        # Initialize
        set_logging()
        device = select_device('0')
        half = device.type != 'cpu'  # half precision only supported on CUDA

        # Load model
        model = attempt_load(weights, map_location=device)  # load FP32 model
        imgsz = check_img_size(imgsz, s=model.stride.max())  # check img_size
        
        #introducing grid size
        gs = model.stride.max()
        #end

        if half:
            model.half()  # to FP16

        # Second-stage classifier
        classify = False
        if classify:
            modelc = load_classifier(name='resnet101', n=2)  # initialize
            modelc.load_state_dict(torch.load('weights/resnet101.pt', map_location=device)['model']).to(device).eval()

        # Set Dataloader
        vid_path, vid_writer = None, None
        
        dataset = LoadImages(source, img_size=imgsz)

        # Get names and colors
        names = model.module.names if hasattr(model, 'module') else model.names
        colors = [[random.randint(0, 255) for _ in range(3)] for _ in names]

        colors = [[217, 175, 78]]

        # Run inference
        t0 = time.time()
        img = torch.zeros((1, 3, imgsz, imgsz), device=device)  # init img
        _ = model(img.half() if half else img) if device.type != 'cpu' else None  # run once
        for path, img, im0s, vid_cap in dataset:
            img = torch.from_numpy(img).to(device)
            img = img.half() if half else img.float()  # uint8 to fp16/32
            img /= 255.0  # 0 - 255 to 0.0 - 1.0
            if img.ndimension() == 3:
                img = img.unsqueeze(0)

            # Inference
            t1 = time_synchronized()
            pred = model(img, augment=True)[0]

            # Apply NMS
            final_pred = non_max_suppression(pred, 0.15, 0.3, classes=0, agnostic=True)
            pred = non_max_suppression(pred, 0.00005, 1, classes=0, agnostic=True)
            t2 = time_synchronized()

            # Apply Classifier
            if classify:
                pred = apply_classifier(pred, modelc, img, im0s)

            # Process detections
            for i, det in enumerate(pred):  # detections per image
                
                p, s, im0 = Path(path), '', im0s.copy()

                imoriginal = im0.copy()
                #plot grid
                numofsquares = int(imgsz/int(gs))
                rowstep = int(im0.shape[0]/numofsquares)
                colstep = int(im0.shape[1]/numofsquares)
                plot_borders(im0, line_color=(0,0,0), thickness=2)
                gridim_solo = im0.copy()
                plot_grid(gridim_solo, pxstep=rowstep, pystep=colstep, line_color=(0,0,0), thickness=2)
                #end plot grid
     
                save_path = str(finalSaveDir + "/" + p.name)
                s += '%gx%g ' % img.shape[2:]  # print string
                gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # normalization gain whwh
                if len(det):
                    # Rescale boxes from img_size to im0 size
                    det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round()

                    # Print results
                    for c in det[:, -1].unique():
                        n = (det[:, -1] == c).sum()  # detections per class
                        s += '%g %ss, ' % (n, names[int(c)])  # add to string
                    
                    # Write results
                    for *xyxy, conf, cls in reversed(det):
                        if save_img or view_img:  # Add bbox to image
                            label = ''#'%s %.2f' % (names[int(cls)], conf)
                            plot_one_box(xyxy, im0, label=label, color=colors[int(cls)], line_thickness=1)
                            
                

                # Print time (inference + NMS)
                print('%sDone. (%.3fs)' % (s, t2 - t1))

                # Save results (image with detections)
                if save_img:
                    cv2.imwrite(save_path + "_original.jpg", imoriginal)
                    cv2.imwrite(save_path, im0)
                    cv2.imwrite(save_path + "_grid.jpg", gridim_solo)
                        


            # SAVE FINAL CROPPED IMAGES
            # Process detections
            for i, det in enumerate(final_pred):  # detections per image
                
                p, s, im0 = Path(path), '', im0s
                im2 = im0.copy() #to use with grid/map
                #background
                numofsquares = int(imgsz/int(gs))
                rowstep = int(im0.shape[0]/numofsquares)
                colstep = int(im0.shape[1]/numofsquares)
                plot_overlay([0,0, im2.shape[1], im2.shape[0]], im2, color=(255, 255, 255), alpha=0.7, pxstep=rowstep, pystep=colstep)
                
                #borders
                plot_borders(im2, line_color=(0,0,0), thickness=2)
                plot_borders(im0, line_color=(0,0,0), thickness=2)

                save_path = str(finalSaveDir + "/" +  p.name)
                s += '%gx%g ' % img.shape[2:]  # print string
                gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # normalization gain whwh
                if len(det):
                    # Rescale boxes from img_size to im0 size
                    det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round()

                    # Print results
                    for c in det[:, -1].unique():
                        n = (det[:, -1] == c).sum()  # detections per class
                        s += '%g %ss, ' % (n, names[int(c)])  # add to string

                    #FUNCTION custom crop
                    CROP = True
                    if CROP:
                        fidx = 0
                        for *xyxy, conf, cls in reversed(det):
                            if save_img or view_img:
                                fidx = fidx + 1
                                crop_object(im0, xyxy, str(finalSaveDir + "/" +  (p.stem + "_cropped_" + str(fidx) + p.suffix)))
                                crop_object(im0, xyxy, str(croppedSaveDir + "/" +  (p.stem + "_cropped_" + str(fidx) + p.suffix)))
                    #END
                    
                    # Write results
                    for *xyxy, conf, cls in reversed(det):
                        if save_img or view_img:  # Add bbox to image
                            label = ''#'%s %.2f' % (names[int(cls)], conf)
                            plot_one_box(xyxy, im0, label=label, color=colors[int(cls)], line_thickness=2)
                            plot_overlay(xyxy, im2, color=colors[int(cls)], alpha=0.7, pxstep=rowstep, pystep=colstep)
                else:
                    cv2.imwrite(save_path + "_not_cropped.jpg", im0)


                gridim = im2.copy()
                plot_grid(gridim, pxstep=rowstep, pystep=colstep, line_color=(0,0,0), thickness=2)
                
                # Print time (inference + NMS)
                print('%sDone. (%.3fs)' % (s, t2 - t1))

                # Save results (image with detections)
                if save_img:
                    cv2.imwrite(save_path + "_map.jpg", gridim)
                    cv2.imwrite(save_path + "_final.jpg", im0)

        if save_txt or save_img:
            s = f"\n{len(list(finalSaveDir.glob('labels/*.txt')))} labels saved to {finalSaveDir + '/' + 'labels'}" if save_txt else ''
            print(f"Results saved to {finalSaveDir}{s}")

        print('Done. (%.3fs)' % (time.time() - t0))

In [ ]:
crop(dataset_dir='/content/dataset/split/', model_path='/content/models/weights/RSGAI_YOLOv3.pt')

# Preview a Cropped Image

In [ ]:
#@title Generate preview

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import glob
lastExp = max(glob.glob(os.path.join('/content/yolov3/runs/detect','*/' )), key=os.path.getmtime)

imgPath = lastExp + 'test/Apple___Black_rot/image (139).JPG'
oringinalImg = mpimg.imread(imgPath + "_original.jpg")
boundingBoxesImg = mpimg.imread(imgPath)
croppedImg = mpimg.imread(imgPath.replace(".JPG", "_cropped_1.JPG"))
gridImg = mpimg.imread(imgPath+ "_grid.jpg")
mapImg = mpimg.imread(imgPath+ "_map.jpg")
finaldetectImg = mpimg.imread(imgPath+ "_final.jpg")

print("Original Image:")
plt.axis('off')
plt.imshow(oringinalImg)
plt.show()

print("Grid:")
plt.axis('off')
plt.imshow(gridImg)
plt.show()

print("Bounding Boxes:")
plt.axis('off')
plt.imshow(boundingBoxesImg)
plt.show()

print("Probability Map:")
plt.axis('off')
plt.imshow(mapImg)
plt.show()

print("Final Detection:")
plt.axis('off')
plt.imshow(finaldetectImg)
plt.show()


print("Cropped Image:")
plt.axis('off')
plt.imshow(croppedImg)
plt.show()

# 4. Exercise III: Segmenting Apple Leaf Images Using a SegNet Model Pretrained on Tomato Leaves

Before you start, make sure to run the "Install and import prequisites", "Download pretrained model", and "Define prerequisite functions" code cells.

Next,  you are required to:

**A.** Loop over the train, test, and val folder under /content/dataset/cropped

**B.** Loop over different classes in each folder

**C.** Loop over images

**D.** Use the pretrained SegNet model to segment each image

**E.** Save images under /content/dataset/segmented/ preserving the train, test, val directory  structure

After that, you should be able to run the "Preview a Segmented Image" cell and see a sample image from the results.

In [ ]:
#@title Install and import prerequisites
!git clone https://github.com/divamgupta/image-segmentation-keras
%cd image-segmentation-keras
from keras_segmentation.models.segnet import segnet
print("Keras and SegNet are loaded")

In [ ]:
#@title Download pretrained model
##########################
### DOWNLOAD THE MODEL ###
##########################
import requests
import os
import zipfile
## FEEL FREE TO CHANGE THESE PARAMETERS
model_URL = "http://faridnakhle.com/pv/models/SegNet.zip"
save_data_to = "/content/models/"
model_file_name = "segnet.zip"
#######################################

if not os.path.exists(save_data_to):
    os.makedirs(save_data_to)

print("Downloading model...")  

r = requests.get(model_URL, stream = True, headers={"User-Agent": "Ready, Steady, Go AI"})
with open(save_data_to + model_file_name, "wb") as file: 
    for block in r.iter_content(chunk_size = 1024):
         if block: 
             file.write(block)

## Extract downloaded zip dataset file
print("Model downloaded")  
print("Extracting files...")

with zipfile.ZipFile(save_data_to + model_file_name, 'r') as zip_dataset:
    zip_dataset.extractall(save_data_to)
print("All done!")  

In [ ]:
#@title Define prerequisite functions

import cv2
import numpy as np
from keras_segmentation.models.segnet import segnet
import glob
import os
from tqdm import tqdm
import six

In [ ]:
def segment(inptDir = ""):

  modelName = "/content/models/RSGAI_SegNet.hdf5"
  model = segnet(n_classes=50 ,  input_height=320, input_width=640)
  model.load_weights(modelName)

  outputDir = "/content/dataset/segmented/"
  inptDirGlob = glob.glob(inptDir + "*")

  ########################
  #### YOUR CODE HERE ####
  ########################
  '''
    1- Loop over the train, test, and val folder under /content/dataset/cropped
    2- Loop over different classes in each folder
    3- Loop over images
    4- Use the SegNet model to segment each image
    5- Save images under /content/dataset/segmented/ preserving the train, test, val directory  structure
  '''
  #############################################
  print("Segmented images are saved in:") 
  print(outputDir)
segment(inptDir="/content/dataset/cropped/")

# Solution

In [ ]:
def segment(inptDir = ""):

  modelName = "/content/models/RSGAI_SegNet.hdf5"
  model = segnet(n_classes=50 ,  input_height=320, input_width=640)
  model.load_weights(modelName)

  outputDir = "/content/dataset/segmented/"

  inptDirGlob = glob.glob(inptDir + "*")
  for setDir in inptDirGlob:

    splitDir = os.path.basename(setDir)
    setClasses = glob.glob(setDir + "/*")

    for setClass in setClasses:

      classDir = os.path.basename(setClass)
      inptFolder = os.path.join(inptDir, splitDir, classDir)
      outputFolder = os.path.join(outputDir, splitDir, classDir)

      if not os.path.exists(outputFolder):
          os.makedirs(outputFolder)

      inps = glob.glob(os.path.join(inptFolder, "*.jpg")) + glob.glob(
          os.path.join(inptFolder, "*.png")) + \
          glob.glob(os.path.join(inptFolder, "*.jpeg"))+ \
          glob.glob(os.path.join(inptFolder, "*.JPG"))
      inps = sorted(inps)

      if len(inps) > 0:

        all_prs = []

        for i, inp in enumerate(tqdm(inps)):
            if outputFolder is None:
                out_fname = None
            else:
                if isinstance(inp, six.string_types):
                    out_fname = os.path.join(outputFolder, os.path.basename(inp))
                else:
                    out_fname = os.path.join(outputFolder, str(i) + ".jpg")

            pr = model.predict_segmentation(
                inp=inp,
                out_fname=out_fname
            )

            img = cv2.imread(inp)
            seg = cv2.imread(out_fname)

            for row in range(0, len(seg)):
                for col in range(0, len(seg[0])):
                    #if np.all(seg[row, col] == [7,47,204]) == False:
                    #    img[row, col] = [0,0,0]
                    
                    if seg[row, col][0] > 50:
                        img[row, col] = [0,0,0]
            all_prs.append(pr)
            cv2.imwrite(out_fname, img)


  print("Segmented images are saved in:") 
  print(outputDir)
segment(inptDir="/content/dataset/cropped/")

# Preview a Segmented Image

In [ ]:
#@title Generate Preview
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
imgPath = '/content/dataset/cropped/test/Apple___Black_rot/image (139)_cropped_1.JPG'
segmemtedPath = '/content/dataset/segmented/test/Apple___Black_rot/image (139)_cropped_1.JPG'

oringinalImg = mpimg.imread(imgPath)
segmentedImage = mpimg.imread(segmemtedPath)

print("Original Image:")
plt.axis('off')
plt.imshow(oringinalImg)
plt.show()

print("Segmented Image:")
plt.axis('off')
plt.imshow(segmentedImage)
plt.show()

# 5. Exercise IV: Descriptive Data Analysis

In this exercise, you are required to generate a bar plot of the data distribution over classes, showing the number of images per class. 

Hint: You need to loop over the classes in the segmented training data folder, then count the images in each. You can use matplotlib to generate the plot.

Based on the above, you can decide whether or not there is a need for data balancing.

In [ ]:
### WRITE YOUR CODE HERE ###

# Solution

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import shutil
import cv2
import matplotlib.pyplot as plt
import seaborn as sns

train_dir = '/content/dataset/segmented/train/'
train_classes = [path for path in os.listdir(train_dir)]
train_imgs = dict([(ID, os.listdir(os.path.join(train_dir, ID))) for ID in train_classes])
train_classes_count = []
for trainClass in train_classes:
  train_classes_count.append(len(train_imgs[trainClass]))

plt.figure(figsize=(15, 10))
g = sns.barplot(x=train_classes, y=train_classes_count)
g.set_xticklabels(labels=train_classes, rotation=30, ha='right')

# 6. Exercise V: Balancing the Dataset

Based on the results of the Descriptive Data Analysis (exercise IV), choose a corresponding data balancing technique to balance the dataset if needed.
 
**NB:** After data balancing, generate the data distribution plot again to analyze the new distribution of classes.

In [ ]:
### YOUR CODE HERE ###

# Solution

It is clear that the dataset is unbalanced. 
We can either downsample the healthy class and augment the cedar apple rust class to average image numbers to around 500 image per class, or we can augment all classes to be balanced.

In our solution, we will augment all classes to 1500 images using Augmentor.

In [ ]:
!pip install Augmentor
import Augmentor
import os

def makedir(path):
    '''
    if path does not exist in the file system, create it
    '''
    if not os.path.exists(path):
        os.makedirs(path)

datasets_root_dir = '/content/dataset/segmented/'
dir = datasets_root_dir + 'train/'
target_dir = dir #same directory as input
makedir(target_dir)

folders = [os.path.join(dir, folder) for folder in next(os.walk(dir))[1]]
target_folders = [os.path.join(target_dir, folder) for folder in next(os.walk(dir))[1]]

requiredNbrOfImages = 1500

for i in range(len(folders)):
    path, dirs, files = next(os.walk(folders[i]))
    nbrOfImages = len(files)
    nbrOfImagesNeeded = requiredNbrOfImages - nbrOfImages
      
    if nbrOfImagesNeeded > 0:
        tfd = target_folders[i]
        print ("saving in " + tfd)
        p = Augmentor.Pipeline(source_directory=folders[i], output_directory=tfd)
        p.rotate(probability=1, max_left_rotation=15, max_right_rotation=15)
        p.flip_left_right(probability=0.5)
        p.skew(probability=1, magnitude=0.2)
        p.flip_left_right(probability=0.5)
        p.shear(probability=1, max_shear_left=10, max_shear_right=10)
        p.flip_left_right(probability=0.5)
        p.sample(nbrOfImagesNeeded)
print("Dataset Augmented!")

# Display Final Data Distribution

In [ ]:
#@title Generate Data Distribution

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import shutil
import cv2
import matplotlib.pyplot as plt
import seaborn as sns

train_dir = '/content/dataset/segmented/train/'
train_classes = [path for path in os.listdir(train_dir)]
train_imgs = dict([(ID, os.listdir(os.path.join(train_dir, ID))) for ID in train_classes])
train_classes_count = []
for trainClass in train_classes:
  train_classes_count.append(len(train_imgs[trainClass]))

plt.figure(figsize=(15, 10))
g = sns.barplot(x=train_classes, y=train_classes_count)
g.set_xticklabels(labels=train_classes, rotation=30, ha='right')


# 7. Exercise VI: Training and Classifying Using the DenseNet-161 Pretrained DCNN algorithm

In this exercise, you are required to implement a DCNN algorithm that uses transfer learning to import a DenseNet-161 pretrained model.
You then need to train the algorithm on the segmented apple training set, and use the validation set to validate the model during training. 

Once training is complete, use the test set to test the overall accuraccy of the model.

In [ ]:
#@title Global Variables Needed

import argparse
import os
import time

import matplotlib.pyplot as plt

import torch
import numpy as np
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from PIL import Image
from collections import OrderedDict
import json

## YOU CAN CHANGE THESE VARIABLES    
EPOCHS = 100
BATCH_SIZE = 20
LEARNING_RATE = 0.0001
data_dir = '/content/dataset/segmented/'
save_checkpoints = True
save_model_to = '/content/output/'
!mkdir /content/output/
IMG_SIZE = 220
NUM_WORKERS = 1
using_gpu = torch.cuda.is_available()
print_every = 300
ARCH = 'densenet161'
######################################################

In [ ]:
### YOUR CODE HERE###

#HINT: Start by create three dataloaders: train_looader, val_loader, and test_loader

''' STEPS:
- Build your model with corresponding layers
- Define loss and optimizer
- Create a function to measure accuracy
- Write the training loop
- Write a function to test with the testing set
'''

# Solution

In [ ]:
def data_loader(root, batch_size=256, workers=1, pin_memory=True):
    traindir = os.path.join(root, 'train')
    valdir = os.path.join(root, 'val')
    testdir = os.path.join(root, 'test')
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

    train_dataset = datasets.ImageFolder(
        traindir,
        transforms.Compose([
            transforms.Resize(size=(IMG_SIZE, IMG_SIZE)),
            transforms.ToTensor(),
            normalize
        ])
    )
    val_dataset = datasets.ImageFolder(
        valdir,
        transforms.Compose([
            transforms.Resize(size=(IMG_SIZE, IMG_SIZE)),
            transforms.ToTensor(),
            normalize
        ])
    )
    test_dataset = datasets.ImageFolder(
        testdir,
        transforms.Compose([
            transforms.Resize(size=(IMG_SIZE, IMG_SIZE)),
            transforms.ToTensor(),
            normalize
        ])
    )

    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=workers,
        pin_memory=pin_memory,
        sampler=None
    )
    val_loader = torch.utils.data.DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=workers,
        pin_memory=pin_memory
    )
    test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=workers,
        pin_memory=pin_memory
    )
    return train_loader, val_loader, test_loader, train_dataset, val_dataset, test_dataset

# Data loading
train_loader, val_loader, test_loader, train_dataset, val_dataset, test_dataset = data_loader(data_dir, BATCH_SIZE, NUM_WORKERS, False)
print("Training Set: " + str(len(train_loader.dataset)))
print("Validation Set: " + str(len(val_loader.dataset)))
print("Testing Set: " + str(len(test_loader.dataset)))

# Freeze parameters so we don't backprop through them
hidden_layers = [10240, 1024]
def make_model(structure, hidden_layers, lr, preTrained):
    if structure=="densenet161":
        model = models.densenet161(pretrained=preTrained)
        input_size = 2208
    else:
        model = models.vgg16(pretrained=preTrained)
        input_size = 25088
    output_size = 102
    for param in model.parameters():
        param.requires_grad = False

    classifier = nn.Sequential(OrderedDict([
                              ('dropout',nn.Dropout(0.5)),
                              ('fc1', nn.Linear(input_size, hidden_layers[0])),
                              ('relu1', nn.ReLU()),
                              ('fc2', nn.Linear(hidden_layers[0], hidden_layers[1])),
                              ('relu2', nn.ReLU()),
                              ('fc3', nn.Linear(hidden_layers[1], output_size)),
                              ('output', nn.LogSoftmax(dim=1))
                              ]))

    model.classifier = classifier
    return model

#############################
## TRANSFER LEARNING MODEL ##
#############################
## Setting the preTrained parameter to true in our make_model function
## will download a pretrained DenseNet-161 model from PyTorch software framework
## then we will train this model on our data, as previously done with the standard DCNN
model = make_model(ARCH, hidden_layers, LEARNING_RATE, True)


# define loss and optimizer
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=LEARNING_RATE)

def cal_accuracy(model, dataloader):
    validation_loss = 0
    accuracy = 0
    for i, (inputs,labels) in enumerate(dataloader):
                optimizer.zero_grad()
                inputs, labels = inputs.to('cuda') , labels.to('cuda')
                model.to('cuda')
                with torch.no_grad():    
                    outputs = model.forward(inputs)
                    validation_loss = criterion(outputs,labels)
                    ps = torch.exp(outputs).data
                    equality = (labels.data == ps.max(1)[1])
                    accuracy += equality.type_as(torch.FloatTensor()).mean()
                    
    validation_loss = validation_loss / len(dataloader)
    accuracy = accuracy /len(dataloader)
    
    return validation_loss, accuracy


def StartTraining(model, image_trainloader, image_valloader, epochs, print_every, criterion, optimizer, device='gpu'):
    epochs = epochs
    print_every = print_every
    steps = 0

    # change to cuda
    model.to('cuda')

    for e in range(epochs):
        running_loss = 0
        for ii, (inputs, labels) in enumerate(image_trainloader):
            steps += 1

            inputs, labels = inputs.to('cuda'), labels.to('cuda')

            optimizer.zero_grad()

            # Forward and backward passes
            outputs = model.forward(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            if steps % print_every == 0:
                model.eval()
                val_loss, train_ac = cal_accuracy(model, image_valloader)
                print("Epoch: {}/{}... | ".format(e+1, epochs),
                      "Loss: {:.4f} | ".format(running_loss/print_every),
                      "Validation Loss {:.4f} | ".format(val_loss),
                      "Accuracy {:.4f}".format(train_ac))

                running_loss = 0
        
        if (save_checkpoints):
            model.class_to_idx = train_dataset.class_to_idx
            state = {
                        'structure' : ARCH,
                        'learning_rate': LEARNING_RATE,
                        'epochs': e,
                        'hidden_layers':hidden_layers,
                        'state_dict':model.state_dict(),
                        'class_to_idx':model.class_to_idx,
                        'optimizer_state_dict': optimizer.state_dict(),
                        'loss': loss
            }
            torch.save(state, save_model_to + ARCH + '_checkpoint_epoch_'+ str(e) +'_acc_'+str(train_ac)+'_.pth')
            torch.save(obj=model, f=save_model_to + ARCH + '_checkpoint_epoch_'+ str(e) +'_acc_'+str(train_ac)+'_MODEL.pth')

StartTraining(model, train_loader, val_loader, EPOCHS, print_every, criterion, optimizer, 'gpu')

## SAVE FINAL MODEL
model.class_to_idx = train_dataset.class_to_idx
state = {
            'structure' : ARCH,
            'learning_rate': LEARNING_RATE,
            'epochs': EPOCHS,
            'hidden_layers':hidden_layers,
            'state_dict':model.state_dict(),
            'class_to_idx':model.class_to_idx
}
torch.save(state, save_model_to +  ARCH +  '_Final.pth')
torch.save(obj=model, f=save_model_to +  ARCH +  '_Final_MODEL.pth')


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.cuda()
@torch.no_grad()
def get_all_preds(model, dataloader):
        
        all_preds = torch.tensor([])
        all_preds = all_preds.to(device)
        all_labels = torch.tensor([])
        all_labels = all_labels.to(device)

        for data, target in dataloader:
            input = data.to(device)
            target = target.to(device)

            with torch.no_grad():
                output = model(input)

            all_preds = torch.cat(
                (all_preds, output)
                ,dim=0
            )
            all_labels = torch.cat(
                (all_labels, target)
                ,dim=0
            )

        return all_preds, all_labels
    
def get_num_correct(preds, labels):
        return preds.argmax(dim=1).eq(labels).sum().item()


with torch.no_grad():
    model.eval()
    test_preds, test_labels = get_all_preds(model,test_loader)

    preds_correct = get_num_correct(test_preds.cuda(), test_labels.cuda())
    print('total correct:', preds_correct)
    print('accuracy:')
    print(((preds_correct / (len(test_loader.dataset))) * 100))

# 8. Exercise VII: Generating Confusion Matrix

Use the confusion_matrix function to generate the confusion matrix, then plot it using matplotlib.

In [ ]:
### YOUR CODE HERE ###

# Solution

In [ ]:
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        #percentage: 
        cm = cm.astype('float') * 100
        # add percentage sign

    mycm = plt.imshow(cm, interpolation='nearest', cmap=cmap)
    mycm.set_clim([0,100])
    cbar = plt.colorbar(mycm, shrink=0.82, ticks=list(range(0, 120, 20)))
    cbar.ax.set_yticklabels(['0', '20', '40', '60', '80', '100'])  # vertically oriented colorbar

    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45,  ha="right")
    
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, str(format(cm[i, j], fmt)) + "%", horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")
        

    plt.rcParams['font.family'] = "sans-serif"
    plt.rcParams['font.sans-serif'] = "Arial"
    plt.rcParams.update({'font.size': 12})
    plt.ylabel('True class', fontsize=17, fontweight='bold')
    plt.xlabel('Predicted class', fontsize=17, fontweight='bold')

import itertools

cmt = torch.zeros(10, 10, dtype=torch.int32) #10 is the number of classes

stacked = torch.stack(
    (
        test_labels
        ,test_preds.argmax(dim=1)
    )
    ,dim=1
)

for p in stacked:
    tl, pl = p.tolist()
    tl = int(tl)
    pl = int(pl)
    cmt[tl, pl] = cmt[tl, pl] + 1

#Plot CM
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(test_labels.cpu(), test_preds.argmax(dim=1).cpu())
print(cm)

plt.figure(figsize=(12, 12))
plot_confusion_matrix(cm, test_dataset.classes, True, 'Confusion matrix', cmap=plt.cm.Blues)
plt.savefig(save_model_to + 'confusionMatrix.eps', format='eps', bbox_inches='tight')
plt.show()

# 9. Exercise VIII: Generating Explanations With LIME

In this exercise you are required to use LIME in order to generate explanations for the classification of the image located under '/content/dataset/segmented/test/Apple___Black_rot/image (475)_cropped_1.JPG'.

Generate the explanations using LIME with the default Quickshift segmentation algorithm, then change the segmentation algorithm to Compact-Watershed and compare the results.

In [ ]:
from lime import lime_image
from skimage import io
from skimage import img_as_ubyte
from skimage.segmentation import mark_boundaries


#@title Define Prerequisite Functions
Pretrainedmodel =  model
def get_PCNN_image(path):
  image = cv2.imread(path)
  image = cv2.resize(image, (226,226))
  return image

PerturbationImgs = []

def batch_predictPDCNN(images):
    Pretrainedmodel.eval()
    batch = torch.stack(tuple(preprocess_transform(i) for i in images), dim=0)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    Pretrainedmodel.to(device)
    batch = batch.to(device)
    logits = Pretrainedmodel(batch)
    probs = F.softmax(logits, dim=1)

    for image in images:
      PerturbationImgs.append(image)

    return probs.detach().cpu().numpy()

def get_preprocess_transform():
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225])     
    transf = transforms.Compose([
        transforms.ToTensor(),
        normalize
    ])    

    return transf    

preprocess_transform = get_preprocess_transform()

explainerPCNN = lime_image.LimeImageExplainer()


In [ ]:
### WRITE YOUR CODE HERE ###

# Solution

In [ ]:
USING QUICKSHIFT:

In [ ]:
image2explain = '/content/dataset/segmented/test/Apple___Cedar_apple_rust/image (268)_cropped_1.JPG'
PCNNimg = get_PCNN_image(image2explain)
PCNNimg = cv2.cvtColor(PCNNimg, cv2.COLOR_BGR2RGB)
explanationPCNN = explainerPCNN.explain_instance(PCNNimg, 
                                         batch_predictPDCNN, top_labels=5, hide_color=0, num_samples=1000)

tempPCNN, maskPCNN = explanationPCNN.get_image_and_mask(explanationPCNN.top_labels[0], positive_only=True, num_features=1, hide_rest=False)





### GENERATE SUPERPIXEL
fig, (ax1) = plt.subplots(1, 1, figsize=(5,5))
ax1.bbox_inches='tight'
ax1.pad_inches = 0
ax1.axis('off')
plt.subplots_adjust(wspace=0, hspace=0)
plt.imshow(mark_boundaries(tempPCNN, explanationPCNN.segments))




###############################
## SUPER PIXEL PERTURPATIONS ##
###############################
from matplotlib import gridspec
## VISUALIZE SOME PERTURBATIONS
# create a figure
fig = plt.figure()
# to change size of subplot's
fig.set_figheight(5)
# set width of each subplot as 8
fig.set_figwidth(15)

# create grid for different subplots
spec = gridspec.GridSpec(ncols=5, nrows=2, wspace=0.1, hspace=0.1)

print("PERTURBATIONS:")
i=0
for perturbationImg in PerturbationImgs:
    p = fig.add_subplot(spec[i])
    p.axis('off')
    p.imshow(perturbationImg)
    i = i + 1
    if i > 9:
      break


In [ ]:
#######################
## SHOW  EXPLANATION ##
#######################
print("FINAL EXPLANATION:")
tempCNNP, maskCNNP = explanationPCNN.get_image_and_mask(explanationPCNN.top_labels[0], positive_only=False, num_features=5, hide_rest=False)
fig, (ax1) = plt.subplots(1, 1, figsize=(5,5))
ax1.bbox_inches='tight'
ax1.pad_inches = 0
ax1.axis('off')
plt.subplots_adjust(wspace=0, hspace=0)
plt.imshow(mark_boundaries(tempCNNP, maskCNNP))

USING COMPACT-WATERSHED:

In [ ]:
from skimage.segmentation import watershed, quickshift, slic
from skimage.color import rgb2gray
from lime.wrappers.scikit_image import SegmentationAlgorithm
from skimage.filters import sobel

def get_PCNN_image(path):
  image = cv2.imread(path)
  image = cv2.resize(image, (226,226))
  return image

PCNNimg = get_PCNN_image(image2explain)
PCNNimg = cv2.cvtColor(PCNNimg, cv2.COLOR_BGR2RGB)

def PCNNcustomSegmentationFunction(image):
  gradient = sobel(rgb2gray(image))
  segments = watershed(gradient, markers=100, compactness=0.001)
  return segments


explainerPCNN = lime_image.LimeImageExplainer()
explanationPCNN = explainerPCNN.explain_instance(PCNNimg, 
                                         batch_predictPDCNN, # classification function
                                         top_labels=5, 
                                         hide_color=0, 
                                         num_samples=1000, segmentation_fn=PCNNcustomSegmentationFunction)


tempPCNN, maskPCNN = explanationPCNN.get_image_and_mask(explanationPCNN.top_labels[0], positive_only=True, num_features=2, hide_rest=False)

### GENERATE SUPERPIXEL
fig, (ax1) = plt.subplots(1, 1, figsize=(5,5))
ax1.bbox_inches='tight'
ax1.pad_inches = 0
ax1.axis('off')
plt.subplots_adjust(wspace=0, hspace=0)
plt.imshow(mark_boundaries(tempPCNN, explanationPCNN.segments))

###############################
## SUPER PIXEL PERTURPATIONS ##
###############################
from matplotlib import gridspec
## VISUALIZE SOME PERTURBATIONS
# create a figure
fig = plt.figure()
# to change size of subplot's
fig.set_figheight(5)
# set width of each subplot as 8
fig.set_figwidth(15)

# create grid for different subplots
spec = gridspec.GridSpec(ncols=5, nrows=2, wspace=0.1, hspace=0.1)

print("PERTURBATIONS:")
i=0
for perturbationImg in PerturbationImgs:
    p = fig.add_subplot(spec[i])
    p.axis('off')
    p.imshow(perturbationImg)
    i = i + 1
    if i > 9:
      break

In [ ]:
###############################
## SHOW POSITIVE EXPLANATION ##
###############################
tempCNNP, maskCNNP = explanationPCNN.get_image_and_mask(explanationPCNN.top_labels[0], positive_only=False, num_features=5, hide_rest=False)
fig, (ax1) = plt.subplots(1, 1, figsize=(5,5))
ax1.bbox_inches='tight'
ax1.pad_inches = 0
ax1.axis('off')
plt.subplots_adjust(wspace=0, hspace=0)
plt.imshow(mark_boundaries(tempCNNP, maskCNNP))